In [ ]:
!pip install stanza
!pip install presidio-analyzer
!pip install presidio-anonymizer

In [ ]:
import stanza
stanza.download("vi")  # Tải mô hình tiếng Việt

In [ ]:
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider

# Cấu hình Stanza cho tiếng Việt
configuration = {
    "nlp_engine_name": "stanza",
    "models": [{"lang_code": "vi", "model_name": "vi"}]
}

# Tạo NLP engine
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine = provider.create_engine()

In [5]:
# Tạo AnalyzerEngine với NLP engine tiếng Việt
analyzer = AnalyzerEngine(
    nlp_engine=nlp_engine,
    supported_languages=["vi"]
)

In [29]:
from presidio_analyzer import PatternRecognizer, Pattern

# Đã có sẵn AnalyzerEngine (analyzer) từ đoạn code trước
# analyzer = AnalyzerEngine()

"""
1. Số điện thoại (Đã có sẵn trong code của bạn, tôi sẽ giữ nguyên và giải thích)
"""
# Định nghĩa regex cho số điện thoại Việt Nam
cccd_pattern = Pattern(
    name="cccd_vi",
    regex=r"\b(0[0-9]{2})([0-9]{9})\b",  # Ví dụ: 012345678901
    score=0.3
)

# Tạo recognizer
cccd_recognizer = PatternRecognizer(
    supported_entity="CCCD",
    patterns=[cccd_pattern],
    supported_language="vi",
    context=["căn cước", "cccd", "căn cước công dân"]
)

# Thêm recognizer vào registry
analyzer.registry.add_recognizer(cccd_recognizer)



"""
1. IP ADDRESS
"""
# Định nghĩa regex cho số điện thoại Việt Nam
cccd_pattern = Pattern(
    name="datetime_vi",
    regex=r"\b(0?[1-9]|[12][0-9]|3[01])[-/.](0?[1-9]|1[0-2])[-/.](\d{4})\b|\b(\d{4})[-/.](0?[1-9]|1[0-2])[-/.](0?[1-9]|[12][0-9]|3[01])\b|\b(0?[1-9]|[12][0-9]|3[01])\s(January|February|March|April|May|June|July|August|September|October|November|December)\s(\d{4})\b",  
    score=0.6
)

# Tạo recognizer
cccd_recognizer = PatternRecognizer(
    supported_entity="DATE_TIME_VN",
    patterns=[cccd_pattern],
    supported_language="vi",
    context=["ngày",  "tháng"]
)

# Thêm recognizer vào registry
analyzer.registry.add_recognizer(cccd_recognizer)

In [33]:
# Văn bản mẫu
text = "Tôi tên Văn Thọ, số điện thoại của tôi là 0823456789 và địa chỉ email là 4pentestonly@gmail.com,  số căn cước công dân cá nhân của tôi là 049202008789, nhà tôi ở Hồ Chí Minh, địa chỉ IP của tôi là 10.2.35.40 và cuối cùng, tôi sinh ngày 23/5/2002"

# Phân tích
results = analyzer.analyze(
    text=text,
    language="vi",
    entities=["PERSON", "PHONE_NUMBER", "EMAIL_ADDRESS", "CCCD","IP_ADDRESS", "LOCATION","URL","NRP" ,"DATE_TIME_VN","CREDIT_CARD", "CRYPTO"]
)

# Sắp xếp kết quả theo thứ tự từ cuối đến đầu để tránh lỗi khi thay thế
results = sorted(results, key=lambda x: x.start, reverse=True)

# Thay thế dữ liệu nhạy cảm bằng label
modified_text = text
for result in results:
    entity_text = modified_text[result.start:result.end]
    entity_label = result.entity_type  # Ví dụ: PHONE_NUMBER, EMAIL, PERSON
    modified_text = modified_text[:result.start] + entity_label + modified_text[result.end:]

# In văn bản đã thay thế
print("\nVăn bản đã thay thế:")
print(modified_text)


Văn bản đã thay thế:
Tôi tên PERSON số điện thoại của tôi là PHONE_NUMBER và địa chỉ email là EMAIL_ADDRESScăn cước công dân cá nhân của tôi là CCCD, nhà tôi ở LOCATIONh, địa chỉ IP của tôi là IP_ADDRESS và cuối cùng, tôi sinh ngày DATE_TIME_VN
